# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [16]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [17]:
pwd

'C:\\Users\\Alvaro Camacho\\Desktop\\Ironhack\\week 7 Remote\\Recommender-Systems\\your-code'

In [18]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [19]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [20]:
data.shape
data.query('CustomerID == 33')

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
527,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1
1017,33,Lindsay,Santana,6509275,417,"Lamb - Pieces, Diced",1
3012,33,Lindsay,Santana,5638266,214,French Pastry - Mini Chocolate,1
3476,33,Lindsay,Santana,4280329,284,Beer - Rickards Red,1
3795,33,Lindsay,Santana,5869069,342,Mangoes,1
...,...,...,...,...,...,...,...
61648,33,Lindsay,Santana,6507033,300,"Beans - Kidney, Canned",1
62883,33,Lindsay,Santana,4150957,319,Towels - Paper / Kraft,1
63171,33,Lindsay,Santana,3056733,187,"Pepper - Black, Whole",1
66163,33,Lindsay,Santana,3179606,296,Wine - Valpolicella Masi,1


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [21]:
df=pd.DataFrame(data.groupby(by=['CustomerID','ProductName']).agg({'Quantity' : 'sum'}).sort_values(by='Quantity',ascending=False))
df = df.reset_index()
df.head()

,CustomerID,ProductName,Quantity
0,90069,Longos - Grilled Salmon With Bbq,92
1,80694,Yeast Dry - Fermipan,84
2,96615,Veal - Eye Of Round,75
3,97063,Bread - French Baquette,75
4,95017,Coffee - Irish Cream,75


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [150]:
dpv = data.pivot_table(index ='ProductName', columns='CustomerID', values='Quantity')
dpv = dpv.fillna(0)
dpv = dpv.T
dpv

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,25.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [151]:
from scipy.spatial.distance import pdist, squareform

distance = pd.DataFrame(squareform(pdist(dpv, 'euclidean')), index=dpv.index, columns=dpv.index)
distance = 1 / (1 + distance)
distance

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [43]:
customer  = 669
top = list(distance[customer].sort_values(ascending=False)[1:6].index)
top
 

[1920, 3305, 3317, 2566, 3535]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [63]:
records = df[df['CustomerID'].isin(top)].reset_index(drop=True)
records

,CustomerID,ProductName,Quantity
0,28867,Veal - Slab Bacon,24
1,28867,Thermometer Digital,16
2,28867,Muffin Chocolate Individual Wrap,16
3,28867,Guinea Fowl,16
4,28867,Ezy Change Mophandle,16
...,...,...,...
309,14501,"Nut - Pistachio, Shelled",4
310,14501,"Nut - Chestnuts, Whole",4
311,14501,Wine - Vineland Estate Semi - Dry,4
312,14501,Wine - Valpolicella Masi,4


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [45]:
ranked = records.groupby(by='ProductName').agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False)
ranked = ranked.reset_index()
ranked

,ProductName,Quantity
0,Cod - Black Whole Fillet,5
1,Pork - Kidney,4
2,Beans - Wax,4
3,"Pepper - Black, Whole",4
4,"Cheese - Boursin, Garlic / Herbs",4
...,...,...
214,Oil - Shortening - All - Purpose,1
215,Apricots - Dried,1
216,Bread - Italian Roll With Herbs,1
217,Onions - Cippolini,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [55]:
dpv = dpv.T
merged = pd.merge(dpv[669],ranked, on='ProductName')
filtered = merged[merged[669]==0] 
filtered.sort_values(by='Quantity', ascending=False).head(5)

,ProductName,669,Quantity
43,Cod - Black Whole Fillet,0.0,5
83,Kellogs Special K Cereal,0.0,4
9,Beans - Wax,0.0,4
126,"Pepper - Black, Whole",0.0,4
179,"Tart Shells - Sweet, 4",0.0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [112]:
recommended = {}
customers = list(set(data['CustomerID']))

for i in customers:
    top = list(distance[i].sort_values(ascending=False)[1:6].index)
    records = df[df['CustomerID'].isin(top)].reset_index(drop=True)
    ranked = records.groupby(by='ProductName').agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False).reset_index()
    merged = pd.merge(dpv[i],ranked, on='ProductName')
    filtered = merged[merged[i]==0] 
    filtered = filtered.sort_values(by='Quantity', ascending=False)[1:6]
    recommended.update({i : list(filtered['ProductName'].values)})
    

In [117]:
dict(recommended) 

{83973: ['Ezy Change Mophandle',
  'Bandage - Fexible 1x3',
  'Cocoa Butter',
  'Pepper - Paprika, Hungarian',
  'Ice - Clear, 300 Lb For Carving'],
 59399: ['Nut - Pistachio, Shelled',
  'Nantucket - Pomegranate Pear',
  'Wine - Gato Negro Cabernet',
  'Bagel - Plain',
  'Snapple - Iced Tea Peach'],
 92168: ['Bandage - Flexible Neon',
  'Beef Wellington',
  'Sauerkraut',
  'Bread - Calabrese Baguette',
  'Orange - Canned, Mandarin'],
 49159: ['Cookies - Assorted',
  'Ezy Change Mophandle',
  'Cheese - Victor Et Berthold',
  'Pasta - Penne, Rigate, Dry',
  'Wine - Magnotta, Merlot Sr Vqa'],
 18441: ['Mushroom - Trumpet, Dry',
  'Soupfoamcont12oz 112con',
  'Island Oasis - Mango Daiquiri',
  'Cheese - Taleggio D.o.p.',
  'Veal - Inside'],
 22536: ['Cheese - Mozzarella',
  'Lettuce - Treviso',
  'Soup - Campbells, Lentil',
  'Tart Shells - Sweet, 4',
  'Lamb - Ground'],
 86028: ['Brandy - Bar',
  'Wine - Valpolicella Masi',
  'Kellogs Special K Cereal',
  'Milk - 1%',
  'Guinea Fowl'],
 

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [129]:

all_recommendations = pd.DataFrame(recommended).T
all_recommendations

,0,1,2,3,4
83973,Ezy Change Mophandle,Bandage - Fexible 1x3,Cocoa Butter,"Pepper - Paprika, Hungarian","Ice - Clear, 300 Lb For Carving"
59399,"Nut - Pistachio, Shelled",Nantucket - Pomegranate Pear,Wine - Gato Negro Cabernet,Bagel - Plain,Snapple - Iced Tea Peach
92168,Bandage - Flexible Neon,Beef Wellington,Sauerkraut,Bread - Calabrese Baguette,"Orange - Canned, Mandarin"
49159,Cookies - Assorted,Ezy Change Mophandle,Cheese - Victor Et Berthold,"Pasta - Penne, Rigate, Dry","Wine - Magnotta, Merlot Sr Vqa"
18441,"Mushroom - Trumpet, Dry",Soupfoamcont12oz 112con,Island Oasis - Mango Daiquiri,Cheese - Taleggio D.o.p.,Veal - Inside
...,...,...,...,...,...
77814,"Wine - Red, Colio Cabernet",Macaroons - Two Bite Choc,Cookie - Dough Variety,Snapple - Iced Tea Peach,"Wine - Red, Harrow Estates, Cab"
32753,Jagermeister,Pail For Lid 1537,Tea - Decaf Lipton,"Chestnuts - Whole,canned",Pie Filling - Cherry
20476,Ice Cream Bar - Hageen Daz To,"Pasta - Detalini, White, Fresh",Wanton Wrap,"Salsify, Organic",Broom - Corn
67582,"Chestnuts - Whole,canned",Cumin - Whole,Cocoa Butter,Cake - Mini Cheesecake,Gloves - Goldtouch Disposable


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [152]:
#dpv = dpv.T
dpv1 = data.pivot_table(index ='ProductName', columns='CustomerID', values='Quantity')
dpv1 = dpv1.fillna(0)
dpv1 = dpv1.T
dpv1
distance = pd.DataFrame(squareform(pdist(dpv1, 'cityblock')), index=dpv.index, columns=dpv.index)
distance += 1
distance

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.0,116.0,94.0,99.0,110.0,104.0,116.0,106.0,108.0,104.0,...,1771.0,1575.0,1644.0,1404.0,1694.0,1627.0,1721.0,1625.0,1973.0,1719.0
200,116.0,1.0,115.0,116.0,133.0,123.0,117.0,121.0,115.0,111.0,...,1772.0,1580.0,1645.0,1403.0,1703.0,1634.0,1714.0,1628.0,1982.0,1734.0
264,94.0,115.0,1.0,108.0,105.0,111.0,115.0,113.0,111.0,111.0,...,1764.0,1574.0,1647.0,1397.0,1689.0,1610.0,1714.0,1618.0,1962.0,1722.0
356,99.0,116.0,108.0,1.0,114.0,112.0,114.0,100.0,114.0,110.0,...,1769.0,1575.0,1644.0,1398.0,1696.0,1621.0,1729.0,1617.0,1963.0,1719.0
412,110.0,133.0,105.0,114.0,1.0,115.0,115.0,105.0,117.0,109.0,...,1780.0,1570.0,1653.0,1403.0,1699.0,1624.0,1732.0,1626.0,1966.0,1714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,1627.0,1634.0,1610.0,1621.0,1624.0,1618.0,1624.0,1616.0,1620.0,1622.0,...,2851.0,2551.0,2826.0,2426.0,2526.0,1.0,2551.0,2551.0,3101.0,3051.0
98069,1721.0,1714.0,1714.0,1729.0,1732.0,1726.0,1728.0,1720.0,1720.0,1718.0,...,3001.0,2801.0,2876.0,2776.0,2626.0,2551.0,1.0,2601.0,3051.0,2801.0
98159,1625.0,1628.0,1618.0,1617.0,1626.0,1620.0,1626.0,1620.0,1616.0,1620.0,...,2851.0,2501.0,2676.0,2626.0,2726.0,2551.0,2601.0,1.0,2851.0,2851.0


In [153]:
customer  = 669
top = list(distance[customer].sort_values(ascending=False)[1:6].index)
top

[91777, 96560, 98185, 97324, 94951]

In [155]:
dpv1 = dpv1.T
recommended2 = {}
customers = list(set(data['CustomerID']))

for i in customers:
    top = list(distance[i].sort_values(ascending=False)[1:6].index)
    records = df[df['CustomerID'].isin(top)].reset_index(drop=True)
    ranked = records.groupby(by='ProductName').agg({'Quantity':'sum'}).sort_values(by='Quantity', ascending=False).reset_index()
    merged = pd.merge(dpv1[i],ranked, on='ProductName')
    filtered = merged[merged[i]==0] 
    filtered = filtered.sort_values(by='Quantity', ascending=False)[1:6]
    recommended2.update({i : list(filtered['ProductName'].values)})

In [156]:
dict(recommended2) 

{83973: ['Juice - Orange',
  'Beef - Montreal Smoked Brisket',
  'Cumin - Whole',
  'Jolt Cola - Electric Blue',
  'Chicken - Soup Base'],
 59399: ['Shrimp - 31/40',
  'Cheese - Brie,danish',
  'Cheese - Cheddarsliced',
  'Juice - Apple Cider',
  'Garlic - Primerba, Paste'],
 92168: ['Scallops - Live In Shell',
  'Pork - Bacon, Double Smoked',
  'Coffee - Hazelnut Cream',
  'Crab - Dungeness, Whole',
  'Crab - Imitation Flakes'],
 49159: ['Shrimp - 31/40',
  'Duck - Breast',
  'Pasta - Penne, Rigate, Dry',
  'Cheese - Brie,danish',
  'Vinegar - Sherry'],
 18441: ['Tahini Paste',
  'Beef - Inside Round',
  'Vinegar - Sherry',
  'Cheese - Brie,danish',
  'Durian Fruit'],
 22536: ['Beef - Inside Round',
  'Cream Of Tartar',
  'Shrimp - 31/40',
  'Mussels - Cultivated',
  'Bread - Hot Dog Buns'],
 86028: ['Lambcasing',
  'Beef - Tenderlion, Center Cut',
  'Broom - Corn',
  'Wine - White, Schroder And Schyl',
  'Cheese - Mix'],
 75791: ['Pasta - Penne, Rigate, Dry',
  'Squid U5 - Thailand',

In [157]:
all_recommendations2 = pd.DataFrame(recommended2).T
all_recommendations2

,0,1,2,3,4
83973,Juice - Orange,Beef - Montreal Smoked Brisket,Cumin - Whole,Jolt Cola - Electric Blue,Chicken - Soup Base
59399,Shrimp - 31/40,"Cheese - Brie,danish",Cheese - Cheddarsliced,Juice - Apple Cider,"Garlic - Primerba, Paste"
92168,Scallops - Live In Shell,"Pork - Bacon, Double Smoked",Coffee - Hazelnut Cream,"Crab - Dungeness, Whole",Crab - Imitation Flakes
49159,Shrimp - 31/40,Duck - Breast,"Pasta - Penne, Rigate, Dry","Cheese - Brie,danish",Vinegar - Sherry
18441,Tahini Paste,Beef - Inside Round,Vinegar - Sherry,"Cheese - Brie,danish",Durian Fruit
...,...,...,...,...,...
77814,Bagel - Plain,"Garlic - Primerba, Paste",Squid U5 - Thailand,"Turnip - White, Organic",Bread - Hot Dog Buns
32753,Beef - Ground Medium,Chicken - Soup Base,Halibut - Fletches,Wine - Chablis 2003 Champs,Cheese - Cheddarsliced
20476,Shrimp - 31/40,Tahini Paste,"Cheese - Brie,danish",Vinegar - Sherry,Beef - Inside Round
67582,Cream Of Tartar,Shrimp - 31/40,Bandage - Flexible Neon,Pastry - Choclate Baked,Durian Fruit


# All the recommendations completely changed!